In [29]:
import re
from collections import *
import pandas as pd
import math

In [ ]:

def filter(text, no_space):
    text = text.lower()
    text = text.replace('ё', 'е').replace('ъ', 'ь')
    text = re.sub(r'[^а-я ]', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()
    if no_space:
        text = text.replace(' ', '')
    return text



def l_freq(text):
    count = Counter(text)
    length = len(text)
    
    for c in count:
        count[c] /=length
    
    l_sorted = dict(sorted(count.items(), key=lambda item: item[1], reverse=True))
    return l_sorted
def bigram_freq(text, one):
    count = Counter()
    step = 1 if one else 2 

    for i in range(0, len(text) - 1, step):
        bigram = text[i:i+2]
        count[bigram] += 1

    total = sum(count.values())
    for bg in count:
        count[bg] /=total

    bg_sorted = dict(sorted(count.items(), key=lambda item: item[1], reverse=True))
    return bg_sorted



def h1(l_freq):
    entropy = 0.0
    for p in l_freq.values():
        if p > 0: 
            entropy -= p * math.log2(p)
    return entropy
def h2(bg_freq):
    h_bigram = 0.0
    for p in bg_freq.values():
        if p > 0:
            h_bigram -= p * math.log2(p)
    return h_bigram /2



def redundancy(H_inf, H0):
    return 1 - (H_inf / H0)



def xlsx1(freq_dict, filename):
    df = pd.DataFrame(freq_dict.items(), columns=["літера", "частота"])
    df.to_excel(filename, index=False)
def xlsx2(freq_dict, filename, alphabet):
    matrix = pd.DataFrame(0.0, index=list(alphabet), columns=list(alphabet))
    for bg in freq_dict:
        row = bg[0]
        col = bg[1]
        matrix.loc[row, col] = freq_dict[bg]
    matrix.to_excel(filename)

In [31]:
with open("Sherlock Holmes.txt", "r", encoding="utf-8") as f:
    text = f.read()

with_spaces = filter(text, no_space=False)
no_spaces = filter(text, no_space=True)

l_freq_space = l_freq(with_spaces)
l_freq_no_space = l_freq(no_spaces)

h1_space = h1(l_freq_space)
h1_no_space = h1(l_freq_no_space)

one_space = bigram_freq(with_spaces, one=True)
one_no_space = bigram_freq(no_spaces, one=True)
two_space = bigram_freq(with_spaces, one=False)
two_no_space = bigram_freq(no_spaces, one=False)


h2_one_space = h2(one_space)
h2_one_no_space = h2(one_no_space)
h2_two_space = h2(two_space)
h2_two_no_space = h2(two_no_space)


alphabet_with_space = 'абвгдежзийклмнопрстуфхцчшщыьэюя '
alphabet_no_space = 'абвгдежзийклмнопрстуфхцчшщыьэюя'

h0_space = math.log2(len(alphabet_with_space))
h0_no_space = math.log2(len(alphabet_no_space))

r1_space = redundancy(h1_space, h0_space)
r1_no_space = redundancy(h1_no_space, h0_no_space)
r2_one_space = redundancy(h2_one_space, h0_space)
r2_one_no_space = redundancy(h2_one_no_space, h0_no_space)
r2_two_space = redundancy(h2_two_space, h0_space)
r2_two_no_space = redundancy(h2_two_no_space, h0_no_space)

print(f"Ентропія літер (з пробілами): {h1_space:.5f}                  R = {r1_space:.5f}")
print(f"Ентропія літер (без пробілів): {h1_no_space:.5f}                 R = {r1_no_space:.5f}")
print(f"\nЕнтропія перехресних біграм (з пробілами): {h2_one_space:.5f}     R = {r2_one_space:.5f}")
print(f"Ентропія неперехресних біграм (з пробілами): {h2_two_space:.5f}   R = {r2_two_space:.5f}")
print(f"\nЕнтропія перехресних біграм (без пробілів): {h2_one_no_space:.5f}    R = {r2_one_no_space:.5f}")
print(f"Ентропія неперехресних біграм (без пробілів): {h2_two_no_space:.5f}  R = {r2_two_no_space:.5f}")


xlsx1(l_freq_space, "Літери з пробілами.xlsx")
xlsx1(l_freq_no_space, "Літери без пробілів.xlsx")
xlsx2(one_space, "Перехресні біграми з пробілами.xlsx", alphabet_with_space)
xlsx2(two_space, "Неперехресні біграми з пробілами.xlsx", alphabet_with_space)
xlsx2(one_no_space, "Перехресні біграми без пробілів.xlsx", alphabet_no_space)
xlsx2(two_no_space, "Неперехресні біграми без пробілів.xlsx", alphabet_no_space)


Ентропія літер (з пробілами): 4.36932                  R = 0.12614
Ентропія літер (без пробілів): 4.45845                 R = 0.10007

Ентропія перехресних біграм (з пробілами): 3.97790     R = 0.20442
Ентропія неперехресних біграм (з пробілами): 3.97756   R = 0.20449

Ентропія перехресних біграм (без пробілів): 4.15282    R = 0.16176
Ентропія неперехресних біграм (без пробілів): 4.15258  R = 0.16180
